In [74]:
from compas.datastructures import Mesh
from compas.datastructures import subdivision as sd
from compas_plotters import MeshPlotter
from compas.geometry import midpoint_point_point, Vector
import random
from statistics import mean

from noise import pnoise1, snoise2
from utilities import draw_compas_mesh, export_obj_by_attribute
import mysubdivision as msd


def plot(m):
    plotter = MeshPlotter(m)
    plotter.draw_edges()
    plotter.draw_faces()
    plotter.show()

mesh = Mesh.from_json('ring_1.json')

draw_compas_mesh(mesh,size=10)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [75]:
def noises(bigY, bigX, octaves=1):
    freq = 16.0 * octaves

    noise_list = []

    for y in range(bigY):
        for x in range(bigX):
            noise_list.append(pnoise2(x / freq, y / freq, octaves))
    
    return noise_list

In [76]:
# Start subdividing

# weird setup
inner_faces = []
for inner_face in mesh.faces_where({'location': 'inner'}):
    inner_faces.append(inner_face)
    
bigY = int(len(inner_faces) * .8)
bigX = len(inner_faces) - bigY

noise_values = noises(bigY, bigX)

print(noise_values[0:100])
    

[0.0, 0.06028175354003906, 0.10894775390625, 0.1387310028076172, 0.146484375, 0.1324939727783203, 0.09979248046875, 0.05347251892089844, 0.0, -0.05347251892089844, -0.09979248046875, -0.1324939727783203, -0.146484375, -0.1387310028076172, -0.10894775390625, -0.06028175354003906, 0.0, 0.06444096565246582, 0.1370391845703125, 0.21798062324523926, 0.3017578125, 0.3800022602081299, 0.4438018798828125, 0.4855034351348877, 0.5, 0.4855034351348877, 0.4438018798828125, 0.3800022602081299, 0.3017578125, 0.21798062324523926, 0.1370391845703125, 0.06444096565246582, 0.0, -0.06236135959625244, -0.12299346923828125, -0.17835581302642822, -0.22412109375, -0.2562481164932251, -0.27179718017578125, -0.26948797702789307, -0.25, -0.21601545810699463, -0.17200469970703125, -0.12375414371490479, -0.07763671875, -0.039624810218811035, -0.01404571533203125, -0.002079606056213379, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.002079606056213379, -0.01404571533203125,

In [77]:
mesh = Mesh.from_json('ring_1.json')

def subdivide_by_ftype(mesh):
    fkeys = list(mesh.faces())
    for fk in fkeys:
        location = mesh.get_face_attribute(fk, 'location')
    
        # OUTER RING MAHIRO 
        if location == 'outer':
            new_keys = msd.taper_face(mesh, fk, height=.5, ratio=0.1)
            
            areas = []
            
            for new_key in new_keys:
                areas.append(mesh.face_area(new_key))
                
            for new_key in new_keys:
                if mesh.face_area(new_key) > mean(areas):
                    mesh.set_face_attribute(new_key, 'location', 'hub')
                else:
                    mesh.set_face_attribute(new_key, 'location', 'circulation')
        
        elif location == 'circulation':
            new_keys = msd.pyramid_face(mesh, fk, height=1)
            
            if random.random()<0.2:
                mesh.set_faces_attribute(new_keys, 'location', 'depot')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'conduits')
        
        elif location == 'conduits':
            new_key = msd.segment_face(mesh, fk, num=10, start_index=0)
            mesh.set_face_attribute(new_key, 'location', 'line')
             
        elif location == 'hub':
            new_keys = msd.segment_face(mesh, fk, num=10, start_index=0)
            mesh.set_faces_attribute(new_keys, 'location', 'part_hub')
        
        elif location == 'construction':
            pass
        
        # INNER RING TETOV
        elif location == 'inner':
            
            noise_value = noise_values.pop()
            
            new_keys = msd.pyramid_face(mesh, fk, height=noise_value)
            
            if fk > noise_value:
                mesh.set_faces_attribute(new_keys, 'location', 'mountain_region')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'sea')
        
        elif location == 'mountain_region':
            
            mount_vertices = mesh.face_vertices(fk)
            mount_vertices_coord = []
            for v in mount_vertices:
                mount_vertices_coord = mesh.vertex_coordinates(v)
            
            face_location = mesh.face_centroid(fk)
            
            direction = Vector.from_start_end(face_location, [0,0,0])
            direction.unitize()
            direction = [i * .3 * pnoise1(fk, 2) for i in direction]

            
            for v in mount_vertices:
                mount_vertices_coord = mesh.vertex_coordinates(v)
                
                new_loc = Vector.sum_vectors([direction, mount_vertices_coord])
            
                mesh.vertex[v]['x'] = new_loc[0]
                mesh.vertex[v]['y'] = new_loc[1]
                mesh.vertex[v]['z'] = new_loc[2]
            
            mesh.set_face_attribute(fk, 'location', 'mountain')
            
        elif location == 'mountain':
            pass
        
        elif location == 'sea':
            pass
               
        # SIDES
        elif location == 'sides':
            pass
        
        else:
            pass
Mesh.face_vertices
for _ in range(5):
    subdivide_by_ftype(mesh)

In [78]:
draw_compas_mesh(mesh)

export_obj_by_attribute('inner.obj', mesh, 'location')